# Voice-of-Customer Data Science Analysis Starter

This notebook provides a starting point for analyzing Voice-of-Customer data collected by the platform. It demonstrates how to:

1. Connect to Astra DB to retrieve data
2. Analyze sentiment trends over time
3. Perform topic modeling on customer feedback
4. Create visualizations for insights
5. Publish insights back to the platform

## Setup and Environment

In [ ]:
# Import required libraries
import os
import json
import uuid
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from dotenv import load_dotenv
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import duckdb
import requests
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Set plot style
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Load environment variables
load_dotenv()

# Configure environment
TENANT_ID = os.getenv("TENANT_ID", "default")
TENANT_NAME = os.getenv("TENANT_NAME", "Default Tenant")
ASTRA_DB_ID = os.getenv("ASTRA_DB_ID")
ASTRA_DB_REGION = os.getenv("ASTRA_DB_REGION")
ASTRA_TOKEN = os.getenv("ASTRA_TOKEN")
ASTRA_KEYSPACE = os.getenv("ASTRA_KEYSPACE", "voc_platform")
MCP_SERVER_URL = os.getenv("MCP_SERVER_URL", "http://mcp-server:3000")

print(f"Analysis for tenant: {TENANT_NAME} ({TENANT_ID})")

## Connect to Astra DB

In [ ]:
# Function to connect to Astra DB
def connect_to_astra():
    # Create a secure connect bundle path
    # In a production environment, download the bundle from Astra DB dashboard
    # For this starter, we'll use the MCP Server as a proxy to Astra DB
    
    cloud_config = {
        'secure_connect_bundle': 'path/to/secure-connect-bundle.zip'
    }
    
    auth_provider = PlainTextAuthProvider(
        'token', 
        ASTRA_TOKEN
    )
    
    try:
        # For local testing, we'll use the MCP server
        print("Using MCP server for data access")
        return None
    except Exception as e:
        print(f"Error connecting to Astra DB: {e}")
        return None

# Alternative function to get data via MCP server API
def get_data_via_mcp(endpoint, params=None):
    url = f"{MCP_SERVER_URL}/api/{endpoint}"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {ASTRA_TOKEN}"
    }
    
    if params is None:
        params = {}
    
    # Add tenant_id if not present
    if 'tenant_id' not in params:
        params['tenant_id'] = TENANT_ID
    
    try:
        response = requests.post(url, headers=headers, json=params)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Error calling MCP API: {e}")
        return None

## Retrieve and Prepare Data

In [ ]:
# Get raw documents
def get_raw_docs(limit=1000, source_type=None, days_back=30):
    params = {
        "limit": limit,
        "days": days_back
    }
    
    if source_type:
        params["source_type"] = source_type
    
    result = get_data_via_mcp("raw-docs/list", params)
    
    if result and 'documents' in result:
        # Convert to DataFrame
        df = pd.DataFrame(result['documents'])
        
        # Parse timestamps
        if 'created_at' in df.columns:
            df['created_at'] = pd.to_datetime(df['created_at'])
        
        # Extract metadata fields
        if 'metadata' in df.columns:
            # Extract sentiment and topics if they exist
            df['sentiment'] = df['metadata'].apply(lambda x: x.get('sentiment', 'neutral') if x else 'neutral')
            df['urgency'] = df['metadata'].apply(lambda x: x.get('urgency', False) if x else False)
            df['topics'] = df['metadata'].apply(lambda x: x.get('topics', []) if x else [])
            df['brands'] = df['metadata'].apply(lambda x: x.get('brands', []) if x else [])
        
        return df
    
    return pd.DataFrame()

# Get sample data
docs_df = get_raw_docs(limit=100)

# Display basic statistics
if not docs_df.empty:
    print(f"Retrieved {len(docs_df)} documents")
    if 'source_type' in docs_df.columns:
        print("\nSource type distribution:")
        print(docs_df['source_type'].value_counts())
    
    if 'sentiment' in docs_df.columns:
        print("\nSentiment distribution:")
        print(docs_df['sentiment'].value_counts())
else:
    print("No documents found or error retrieving data")
    
    # Create sample data for demonstration
    print("Creating sample data for demonstration purposes")
    
    # Sample data generation
    np.random.seed(42)
    dates = pd.date_range(end=datetime.now(), periods=100, freq='D')
    
    sentiments = np.random.choice(['positive', 'neutral', 'negative'], size=100, p=[0.3, 0.5, 0.2])
    
    source_types = np.random.choice(['website', 'review_site', 'social_media', 'serp'], size=100, p=[0.4, 0.3, 0.2, 0.1])
    
    topics_list = [
        ['shipping', 'delivery'], 
        ['price', 'cost'], 
        ['quality', 'durability'],
        ['customer service', 'support'],
        ['ease of use', 'user interface'],
        ['features', 'functionality'],
        ['reliability', 'uptime'],
        ['performance', 'speed']
    ]
    
    topics = [np.random.choice(topics_list, size=np.random.randint(1, 3)).tolist() for _ in range(100)]
    
    brands = [['Brand A', 'Brand B'] if i % 5 == 0 else ['Brand A'] if i % 3 == 0 else ['Brand C'] for i in range(100)]
    
    urgency = np.random.choice([True, False], size=100, p=[0.15, 0.85])
    
    docs_df = pd.DataFrame({
        'doc_id': [str(uuid.uuid4()) for _ in range(100)],
        'tenant_id': TENANT_ID,
        'source_type': source_types,
        'created_at': dates,
        'sentiment': sentiments,
        'urgency': urgency,
        'topics': topics,
        'brands': brands,
        'content': ['Sample content ' + str(i) for i in range(100)]
    })
    
    print("\nSample data source type distribution:")
    print(docs_df['source_type'].value_counts())
    
    print("\nSample data sentiment distribution:")
    print(docs_df['sentiment'].value_counts())

## Sentiment Analysis Over Time

In [ ]:
# Time series sentiment analysis
def plot_sentiment_over_time(df):
    if df.empty or 'created_at' not in df.columns or 'sentiment' not in df.columns:
        print("Cannot create time series: Missing required columns")
        return
    
    # Resample by day and count sentiments
    sentiment_counts = df.set_index('created_at').\
        groupby([pd.Grouper(freq='D'), 'sentiment']).size().unstack(fill_value=0)
    
    # Ensure all sentiment columns exist
    for sentiment in ['positive', 'neutral', 'negative']:
        if sentiment not in sentiment_counts.columns:
            sentiment_counts[sentiment] = 0
    
    # Calculate moving averages
    window = 7
    rolling_sentiments = sentiment_counts.rolling(window=window).mean()
    
    # Plot
    plt.figure(figsize=(16, 8))
    
    # Plot raw sentiment counts
    plt.subplot(2, 1, 1)
    sentiment_counts.plot(ax=plt.gca(), kind='bar', stacked=True, 
                          color={'positive': 'green', 'neutral': 'grey', 'negative': 'red'})
    plt.title(f'Daily Sentiment Distribution for {TENANT_NAME}')
    plt.xlabel('Date')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.legend(title='Sentiment')
    
    # Plot rolling averages
    plt.subplot(2, 1, 2)
    rolling_sentiments.plot(ax=plt.gca(), kind='line', 
                           color={'positive': 'green', 'neutral': 'grey', 'negative': 'red'})
    plt.title(f'{window}-Day Rolling Average Sentiment')
    plt.xlabel('Date')
    plt.ylabel('Average Count')
    plt.legend(title='Sentiment')
    
    plt.tight_layout()
    plt.show()

# Plot sentiment over time
plot_sentiment_over_time(docs_df)

## Topic Analysis

In [ ]:
# Extract and analyze topics
def analyze_topics(df):
    if df.empty or 'topics' not in df.columns:
        print("Cannot analyze topics: Missing required column")
        return
    
    # Flatten topics list
    all_topics = []
    for topics_list in df['topics']:
        if isinstance(topics_list, list):
            all_topics.extend(topics_list)
        elif isinstance(topics_list, str):
            # Handle case where topics might be a string representation of a list
            try:
                parsed = json.loads(topics_list.replace("'", "\""))
                if isinstance(parsed, list):
                    all_topics.extend(parsed)
            except:
                all_topics.append(topics_list)
    
    # Count topic frequencies
    topic_counts = pd.Series(all_topics).value_counts()
    
    # Plot top topics
    plt.figure(figsize=(12, 6))
    topic_counts.head(15).plot(kind='barh', color='skyblue')
    plt.title(f'Top 15 Topics Mentioned - {TENANT_NAME}')
    plt.xlabel('Frequency')
    plt.ylabel('Topic')
    plt.tight_layout()
    plt.show()

# Analyze topics
analyze_topics(docs_df)

## Source Analysis

In [ ]:
# Analyze data by source
def analyze_by_source(df):
    if df.empty or 'source_type' not in df.columns:
        print("Cannot analyze by source: Missing required column")
        return
    
    # Count by source type
    source_counts = df['source_type'].value_counts()
    
    # Plot source distribution
    plt.figure(figsize=(10, 6))
    source_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90, cmap='Pastel1')
    plt.title(f'Distribution of Data Sources - {TENANT_NAME}')
    plt.ylabel('')
    plt.tight_layout()
    plt.show()

# Analyze by source
analyze_by_source(docs_df)

## Publishing Insights to Platform

In [ ]:
# Function to publish an insight to the platform
def publish_insight(title, description, figure=None, source_ids=None, metadata=None):
    """Publish an insight to the platform."""
    
    # Create insight data
    insight_data = {
        "tenant_id": TENANT_ID,
        "insight_id": str(uuid.uuid4()),
        "title": title,
        "description": description,
        "source_ids": source_ids or [],
        "created_at": datetime.now().isoformat(),
        "metadata": metadata or {}
    }
    
    # Save figure if provided
    if figure is not None:
        # In a production environment, this would save to S3/MinIO
        # For this demo, we'll just note it
        print(f"Would save figure for insight: {title}")
        
    # Send insight to MCP server
    result = get_data_via_mcp("insights/create", insight_data)
    
    if result and result.get("success"):
        print(f"✅ Published insight: {title}")
        return True
    else:
        print(f"❌ Failed to publish insight: {title}")
        return False

# Example: Publish a sample insight
# publish_insight(
#     title="Sentiment Trend Analysis: Q1 2024",
#     description="Analysis of customer sentiment trends over Q1 2024 shows a 15% increase in positive sentiment related to product quality, while concerns about shipping times have decreased by 8%.",
#     metadata={
#         "period": "Q1 2024",
#         "key_findings": ["Positive sentiment increase", "Shipping concerns decreased"],
#         "recommendation": "Continue product quality improvements"
#     }
# )